# **Calcular Costos de Tamaño**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

pd.options.display.float_format = '{:.2f}'.format

In [48]:
bdd_ventas = pd.read_excel("BDD_Bodegas.xlsx", sheet_name="PROY 10 AÑOS")
bdd_ventas['Fecha'] = pd.to_datetime(bdd_ventas['Fecha'])
bdd_ventas['Año-Sem'] = bdd_ventas['Fecha'].dt.strftime('%Y-%U')
bdd_ventas = bdd_ventas.loc[:, ['ID Cliente', 'Cantidad', 'ID Bodega Despacho', 'Año-Sem']]
bdd_ventas = bdd_ventas.rename(columns={'ID Bodega Despacho': 'Bodega Asignada'})
bdd_ventas = bdd_ventas.loc[:, ['ID Cliente', 'Cantidad', 'Bodega Asignada', 'Año-Sem']].reset_index(drop=True)
bdd_ventas


,ID Cliente,Cantidad,Bodega Asignada,Año-Sem
0,13,854.82,2,2031-38
1,17,33.65,4,2031-38
2,21,20.19,6,2031-38
3,39,562.03,2,2031-38
4,47,33.65,9,2031-38
...,...,...,...,...
13981,2309,33.65,6,2032-38
13982,2589,33.65,3,2032-38
13983,2603,48.80,3,2032-38
13984,2732,42.07,7,2032-38


In [49]:
df_bodegas = bdd_ventas.groupby(['Bodega Asignada', 'Año-Sem'])['Cantidad'].sum().reset_index()

In [50]:
df_bodegas

,Bodega Asignada,Año-Sem,Cantidad
0,1,2031-38,15954.66
1,1,2031-39,11896.17
2,1,2031-40,7284.47
3,1,2031-41,20917.82
4,1,2031-42,7418.25
...,...,...,...
528,10,2032-34,1606.99
529,10,2032-35,2589.70
530,10,2032-36,4444.05
531,10,2032-37,336.54


In [51]:
promedio_por_bodega = df_bodegas.groupby('Bodega Asignada')['Cantidad'].mean()
promedio_bodegas = dict()
for i in range(len(promedio_por_bodega)):
    promedio_bodegas[promedio_por_bodega.index[i]] = promedio_por_bodega.iloc[i]

costo_tpte = 1200 # Costo por kilómetro recorrido por camión
costo_posicion = 36700 # Costo fijo de posición por producto en una bodega
distancias_Stgo_a = {1: 452.06, 2: 116.03, 3: 83.45, 4: 155.65, 5: 217.80, 6: 349.07, 7: 476.87, 8: 535.40, 9: 709.86, 10: 991.26}
distancias_Stgo_f = {1: 452.06, 2: 116.03, 3: 83.45, 4: 155.65, 5: 217.80, 6: 349.07, 7: 476.87, 8: 535.40, 9: 709.86, 10: 991.26}

In [52]:
def calcular_promedios(df_bodegas):
    promedio_por_bodega = df_bodegas.groupby('Bodega Asignada')['Cantidad'].mean()
    promedio_bodegas = dict()
    for i in range(len(promedio_por_bodega)):
        promedio_bodegas[promedio_por_bodega.index[i]] = promedio_por_bodega.iloc[i]
    return promedio_bodegas

In [53]:
len(df_bodegas)

533

In [54]:
promedio_bodegas = calcular_promedios(df_bodegas)
promedio_bodegas

{1: 11317.900721143957,
 2: 3320.0799369621895,
 3: 16303.123994537618,
 4: 8446.34692855729,
 5: 13023.833937115172,
 6: 11850.608375305103,
 7: 6014.296652876393,
 8: 9155.514310652612,
 9: 5836.458610113825,
 10: 2340.3697593520533}

In [55]:
def calcular_promedios(df_bodegas):
    promedio_por_bodega = df_bodegas.groupby('Bodega Asignada')['Cantidad'].mean()
    promedio_bodegas = dict()
    for i in range(len(promedio_por_bodega)):
        promedio_bodegas[promedio_por_bodega.index[i]] = promedio_por_bodega.iloc[i]
    return promedio_bodegas

def tasa_ocupacion (demanda, promedio_bodegas, numero_bodega, delta):
    tamaño = promedio_bodegas[numero_bodega]*delta
    bdd = demanda[demanda["Bodega Asignada"] == numero_bodega]
    suma = 0
    for i in range(len(bdd)):
        if bdd.iloc[i, 2] >= tamaño:
            suma += 1
        else:
            suma += bdd.iloc[i, 2] / tamaño
    promedio = suma / len(bdd)
    return promedio

def semanas_sobreocupacion(demanda, promedio_bodegas, numero_bodega, delta):
    tamaño = promedio_bodegas[numero_bodega]*delta
    bdd = demanda[demanda["Bodega Asignada"] == numero_bodega]
    suma = 0
    camiones_a_pedir = 0
    for i in range(len(bdd)):
        if bdd.iloc[i, 2] > tamaño:
            suma += 1
        camiones_a_pedir += math.ceil(bdd.iloc[i,2]/tamaño)
    porcentaje = round(suma/len(bdd) * 100, 1)
    return suma, f"{porcentaje}%", camiones_a_pedir

def semanas_sobreocupacion_sim(demanda, promedio_bodegas, numero_bodega, delta):
    tamaño = promedio_bodegas[numero_bodega]*delta
    bdd = demanda[demanda["Bodega Asignada"] == numero_bodega]
    suma = 0
    camiones_a_pedir = 0
    for i in range(len(bdd)):
        if bdd.iloc[i, 2] > tamaño:
            suma += 1
        camiones_a_pedir += math.ceil(bdd.iloc[i,2]/tamaño)
    porcentaje = suma/len(bdd)
    return suma, porcentaje, camiones_a_pedir

def analisis_tamaño(demanda, promedio_bodegas, numero_bodega, delta, caso, N):
    resultados = []
    distancias_Stgo_a = {1: 452.06, 2: 116.03, 3: 83.45, 4: 155.65, 5: 217.80, 6: 349.07, 7: 476.87, 8: 535.40, 9: 709.86, 10: 991.26}
    distancias_Stgo_f = {1: 452.06, 2: 116.03, 3: 83.45, 4: 155.65, 5: 217.80, 6: 349.07, 7: 476.87, 8: 535.40, 9: 709.86, 10: 991.26}
    if caso == 'Actual':
        distancias = distancias_Stgo_a
    else:
        distancias = distancias_Stgo_f
    tamaño = promedio_bodegas[numero_bodega]*delta
    tasa = tasa_ocupacion(demanda, promedio_bodegas, numero_bodega, delta)
    semanas, porcentaje, camiones = semanas_sobreocupacion(demanda, promedio_bodegas, numero_bodega, delta)
    costo_fijo = 36700*tamaño/1000000
    costo_tpte = 1200*distancias[numero_bodega]*camiones/1000000
    costo_tpte = costo_tpte/N
    costo_total = costo_fijo + costo_tpte
    resultados.append([delta, f"{round(tasa * 100, 2)}%", tamaño, (f'{semanas} semanas, {porcentaje} del total'), costo_fijo, costo_tpte, costo_total])
    return resultados[0]

def analisis_tamaño_sim(demanda, promedio_bodegas, numero_bodega, delta, caso, N):
    resultados = []
    distancias_Stgo_a = {1: 452.06, 2: 116.03, 3: 83.45, 4: 155.65, 5: 217.80, 6: 349.07, 7: 476.87, 8: 535.40, 9: 709.86, 10: 991.26}
    distancias_Stgo_f = {1: 452.06, 2: 116.03, 3: 83.45, 4: 155.65, 5: 217.80, 6: 349.07, 7: 476.87, 8: 535.40, 9: 709.86, 10: 991.26}
    if caso == 'Actual':
        distancias = distancias_Stgo_a
    else:
        distancias = distancias_Stgo_f
    tamaño = promedio_bodegas[numero_bodega]*delta
    tasa = tasa_ocupacion(demanda, promedio_bodegas, numero_bodega, delta)
    semanas, porcentaje, camiones = semanas_sobreocupacion_sim(demanda, promedio_bodegas, numero_bodega, delta)
    costo_fijo = 36700*tamaño/1000000
    costo_tpte = 1200*distancias[numero_bodega]*camiones/1000000
    costo_tpte = costo_tpte/N
    costo_total = costo_fijo + costo_tpte
    resultados.append([delta, tasa, tamaño, porcentaje, costo_fijo, costo_tpte, costo_total])
    return resultados[0]

def costo_minimo(df_bodegas, promedio_bodegas, numero_bodega, caso, N):
    delta = 0
    resultado_optimo = None
    min = math.inf
    optimo = False
    while optimo == False:
        delta += 0.01
        resultado = analisis_tamaño(df_bodegas, promedio_bodegas, numero_bodega, delta, caso, N)
        costo = resultado[6]
        if costo < min:
            min = costo
            resultado_optimo = resultado
        else:
            optimo = True
    return resultado_optimo

def costo_minimo_sim(df_bodegas, promedio_bodegas, numero_bodega, caso, N):
    delta = 0
    resultado_optimo = None
    min = math.inf
    optimo = False
    while optimo == False:
        delta += 0.01
        resultado = analisis_tamaño_sim(df_bodegas, promedio_bodegas, numero_bodega, delta, caso, N)
        costo = resultado[6]
        if costo < min:
            min = costo
            resultado_optimo = resultado
        else:
            optimo = True
    return resultado_optimo

def tamaños_optimos(df_bodegas, promedio_bodegas, caso):
    N = 1
    #N = (len(df_bodegas)/3)/54
    resultados = list()
    for bodega in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
        resultado = costo_minimo(df_bodegas, promedio_bodegas, bodega, caso, N)
        resultado.insert(0, bodega)
        resultados.append(resultado)
    df_resultados = pd.DataFrame(resultados, columns=["Bodega", "Delta", "Ocupación", "Tamaño", "Sobreocupación", "CF (MM)", "CTpte (MM)", "CT (MM)"])
    return df_resultados

def tamaños_optimos_sim(df_bodegas, promedio_bodegas, caso):
    N = 1
    #N = (len(df_bodegas)/3)/54
    resultados = list()
    for bodega in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
        resultado = costo_minimo_sim(df_bodegas, promedio_bodegas, bodega, caso, N)
        resultado.insert(0, bodega)
        resultados.append(resultado)
    df_resultados = pd.DataFrame(resultados, columns=["Bodega", "Delta", "Ocupación", "Tamaño", "Sobreocupación", "CF (MM)", "CTpte (MM)", "CT (MM)"])
    return df_resultados

In [56]:
promedio_bodegas = calcular_promedios(df_bodegas)
costos_optimos = tamaños_optimos_sim(df_bodegas, promedio_bodegas, 'Futuro')
costos_optimos

,Bodega,Delta,Ocupación,Tamaño,Sobreocupación,CF (MM),CTpte (MM),CT (MM)
0,1,0.28,0.99,3169.01,0.93,116.30,117.72,234.02
1,2,0.22,0.99,730.42,0.96,26.81,36.20,63.01
2,3,0.10,0.95,1630.31,0.91,59.83,56.68,116.51
3,4,0.17,0.93,1435.88,0.93,52.70,63.88,116.58
4,5,0.16,0.99,2083.81,0.90,76.48,90.95,167.43
5,6,0.22,0.87,2607.13,0.70,95.68,111.00,206.69
6,7,0.33,0.91,1984.72,0.81,72.84,108.15,180.99
7,8,0.27,0.81,2471.99,0.70,90.72,143.27,234.00
8,9,0.29,0.89,1692.57,0.83,62.12,178.03,240.15
9,10,0.60,0.77,1404.22,0.56,51.53,139.17,190.71


In [20]:
b = tamaños_optimos(df_bodegas, promedio_bodegas, 'Actual')
b

,Delta,Ocupación,Tamaño,Sobreocupación,CF (MM),CTpte (MM),CT (MM)
0,0.42,92.84%,1035.02,"234 semanas, 78.0% del total",37.99,63.88,101.86
1,0.15,97.82%,1779.72,"273 semanas, 91.3% del total",65.32,77.07,142.39
2,0.39,87.18%,2000.46,"212 semanas, 70.7% del total",73.42,98.37,171.79


In [27]:
b20 = tamaños_optimos(df_bodegas, promedio_bodegas, 'Actual')
b20

,Delta,Ocupación,Tamaño,Sobreocupación,CF (MM),CTpte (MM),CT (MM)
0,0.48,90.54%,1182.88,"214 semanas, 71.3% del total",43.41,68.45,111.87
1,0.17,96.85%,2017.01,"262 semanas, 87.6% del total",74.02,82.18,156.21
2,0.39,87.18%,2000.46,"212 semanas, 70.7% del total",73.42,118.04,191.46


In [8]:
c = tamaños_optimos(df_bodegas, promedio_bodegas, 'Futuro')
c

,Delta,Ocupación,Tamaño,Sobreocupación,CF (MM),CTpte (MM),CT (MM)
0,0.31,99.61%,1909.98,"52 semanas, 96.3% del total",70.10,89.85,159.95
1,0.08,96.97%,4660.66,"52 semanas, 96.3% del total",171.05,160.22,331.27
2,0.23,94.92%,4398.43,"49 semanas, 90.7% del total",161.42,188.11,349.54


In [33]:
c20 = tamaños_optimos(df_bodegas, promedio_bodegas, 'Futuro')
c20

,Delta,Ocupación,Tamaño,Sobreocupación,CF (MM),CTpte (MM),CT (MM)
0,0.31,99.61%,1909.98,"52 semanas, 96.3% del total",70.10,107.82,177.92
1,0.08,96.97%,4660.66,"52 semanas, 96.3% del total",171.05,192.27,363.32
2,0.23,94.92%,4398.43,"49 semanas, 90.7% del total",161.42,225.74,387.16


In [58]:
cn = tamaños_optimos_sim(df_bodegas, promedio_bodegas, 'Futuro')
cn

,Bodega,Delta,Ocupación,Tamaño,Sobreocupación,CF (MM),CTpte (MM),CT (MM)
0,1,0.31,1.00,1909.98,0.96,70.10,89.85,159.95
1,4,0.08,0.97,4660.66,0.96,171.05,160.22,331.27
2,8,0.23,0.95,4398.43,0.91,161.42,188.11,349.54


In [46]:
cn = tamaños_optimos(df_bodegas, promedio_bodegas, 'Futuro')
cn

,Bodega,Delta,Ocupación,Tamaño,Sobreocupación,CF (MM),CTpte (MM),CT (MM)
0,1,0.31,99.61%,1909.98,"52 semanas, 96.3% del total",70.10,89.85,159.95
1,4,0.08,96.97%,4660.66,"52 semanas, 96.3% del total",171.05,160.22,331.27
2,8,0.23,94.92%,4398.43,"49 semanas, 90.7% del total",161.42,188.11,349.54
